In [5]:
import pandas as pd
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
df = pd.read_csv('train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [9]:
# Test du model de tweet avec word2vec
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [39]:
import gensim.models
sentences = df['text'].apply(lambda x: x.split()).tolist()
model = gensim.models.Word2Vec(sentences=sentences)

In [40]:
import numpy as np
vec_list = []
for word in df['text'].str.split().explode().unique():
    if word in model.wv.key_to_index:  # Vérifiez si le mot est présent dans le vocabulaire
        vec_list.append(model.wv[word])
    else:
        vec_list.append(np.zeros(model.vector_size))  # Affectez un vecteur zéro pour les mots manquants
vec_df = np.array(vec_list)


In [41]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

'word #0/3000000 is </s>'
'word #1/3000000 is in'
'word #2/3000000 is for'
'word #3/3000000 is that'
'word #4/3000000 is is'
'word #5/3000000 is on'
'word #6/3000000 is ##'
'word #7/3000000 is The'
'word #8/3000000 is with'
'word #9/3000000 is said'


In [42]:
# The default value of vector_size is 100.
model = gensim.models.Word2Vec(sentences, vector_size=200)

In [43]:
# Train the model with sentence
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(307995, 567305)

In [44]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

3123157.25


In [51]:
# Test du model de tweet avec fasttext
from pprint import pprint as print
from gensim.models.fasttext import FastText


In [56]:
# Set file names for train and test data

model = FastText(vector_size=100)
model.build_vocab(corpus_iterable=sentences)
model.train(
    corpus_iterable=sentences, epochs=model.epochs,
    total_examples=len(sentences), total_words=model.corpus_total_words,
)

(308214, 567305)